In [1]:
import numpy as np
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.nn.init as init
import torch.utils.data as data
import torch.utils.data.dataset as dataset
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.utils as v_utils
import matplotlib.pyplot as plt
import cv2
import math
from collections import OrderedDict
import copy
import time
from model.utils import DataLoader
from sklearn.metrics import roc_auc_score
from utils import *
import random

import argparse

In [2]:
# Define the dictionary containing your arguments
args_dict = {
    'gpus': '0',
    'batch_size': 2,
    'test_batch_size': 1,
    'epochs': 10,
    'loss_compact': 0.1,
    'loss_separate': 0.1,
    'h': 128,
    'w': 128,
    'c': 3,
    'lr': 2e-4,
    'method': 'pred',
    't_length': 5,
    'fdim': 512,
    'mdim': 512,
    'msize': 10,
    'num_workers': 2,
    'num_workers_test': 1,
    'dataset_type': 'ped2',
    'dataset_path': './dataset',
    'exp_dir': 'log'
}

In [3]:
# Create an ArgumentParser instance
parser = argparse.ArgumentParser(description="MNAD")

# Add arguments to the parser
for arg, value in args_dict.items():
    parser.add_argument(f'--{arg}', type=type(value), default=value)

# Parse the arguments
args = parser.parse_args([])

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
if args.gpus is None:
    gpus = "0"
    os.environ["CUDA_VISIBLE_DEVICES"]= gpus
else:
    gpus = ""
    for i in range(len(args.gpus)):
        gpus = gpus + args.gpus[i] + ","
    os.environ["CUDA_VISIBLE_DEVICES"]= gpus[:-1]

In [5]:
torch.backends.cudnn.enabled = True

In [6]:
train_folder = args.dataset_path+"/"+args.dataset_type+"/training/frames"
test_folder = args.dataset_path+"/"+args.dataset_type+"/testing/frames"

In [7]:
# Loading dataset
train_dataset = DataLoader(train_folder, transforms.Compose([
             transforms.ToTensor(),          
             ]), resize_height=args.h, resize_width=args.w, time_step=args.t_length-1)

test_dataset = DataLoader(test_folder, transforms.Compose([
             transforms.ToTensor(),            
             ]), resize_height=args.h, resize_width=args.w, time_step=args.t_length-1)

In [8]:
transform = transforms.Compose([
    transforms.ToTensor()
])

# Initialize your custom DataLoader
resize_height = 128  # Example resize height
resize_width = 128   # Example resize width
time_step = 4        # Example time step
num_pred = 1         # Example number of predictions
custom_dataloader = DataLoader(train_folder, transform, resize_height, resize_width, time_step, num_pred)

# Now, you can iterate over your DataLoader
#for batch in enumerate(custom_dataloader):
    # Your batch processing logic here
#    print(batch)  # Example: printing the shape of the batch

In [9]:
train_size = len(train_dataset)
test_size = len(test_dataset)

train_size, test_size

(408, 352)

In [10]:
train_batch = data.DataLoader(train_dataset, batch_size = args.batch_size, 
                              shuffle=True, num_workers=args.num_workers, drop_last=True)
test_batch = data.DataLoader(test_dataset, batch_size = args.test_batch_size, 
                             shuffle=False, num_workers=args.num_workers_test, drop_last=False)

In [11]:
# Model setting
assert args.method == 'pred' or args.method == 'recon', 'Wrong task name'
if args.method == 'pred':
    from model.final_future_prediction_with_memory_spatial_sumonly_weight_ranking_top1 import *
    model = convAE(args.c, args.t_length, args.msize, args.fdim, args.mdim)
else:
    from model.Reconstruction import *
    model = convAE(args.c, memory_size = args.msize, feature_dim = args.fdim, key_dim = args.mdim)
params_encoder =  list(model.encoder.parameters()) 
params_decoder = list(model.decoder.parameters())
params = params_encoder + params_decoder
optimizer = torch.optim.Adam(params, lr = args.lr)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max =args.epochs)
# model.cuda()
model.cuda()

convAE(
  (encoder): Encoder(
    (moduleConv1): Sequential(
      (0): Conv2d(12, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
    (modulePool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (moduleConv2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
    (modulePool2): MaxPool2d(kernel_size=2, stride=2,

In [12]:
log_dir = os.path.join('./exp', args.dataset_type, args.method, args.exp_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
orig_stdout = sys.stdout
f = open(os.path.join(log_dir, 'log.txt'),'w')
sys.stdout= f

In [13]:
for j, (imgs) in enumerate(train_batch):
    print("Working")

In [14]:
loss_func_mse = nn.MSELoss(reduction='none')

In [15]:
m_items = F.normalize(torch.rand((args.msize, args.mdim), dtype=torch.float), dim=1).cuda()

In [16]:
for epoch in range(args.epochs):
    labels_list = []
    model.train()
    
    start = time.time()
    for j, (imgs) in enumerate(train_batch):
        # Move data to CPU
        # torch.cuda.empty_cache()
        imgs = Variable(imgs).cuda()
        print("inside")

        if args.method == 'pred':
            outputs, _, _, m_items, softmax_score_query, softmax_score_memory, separateness_loss, compactness_loss = model.forward(imgs[:, 0:12], m_items, True)
        else:
            outputs, _, _, m_items, softmax_score_query, softmax_score_memory, separateness_loss, compactness_loss = model.forward(imgs, m_items, True)

        optimizer.zero_grad()
        
        if args.method == 'pred':
            loss_pixel = torch.mean(loss_func_mse(outputs, imgs[:, 12:]))
        else:
            loss_pixel = torch.mean(loss_func_mse(outputs, imgs))
            
        loss = loss_pixel + args.loss_compact * compactness_loss + args.loss_separate * separateness_loss
        loss.backward(retain_graph=True)
        optimizer.step()
        print("End one enumerator")
        
    scheduler.step()
    
    print('----------------------------------------')
    print('Epoch:', epoch + 1)
    if args.method == 'pred':
        print('Loss: Prediction {:.6f}/ Compactness {:.6f}/ Separateness {:.6f}'.format(loss_pixel.item(), compactness_loss.item(), separateness_loss.item()))
    else:
        print('Loss: Reconstruction {:.6f}/ Compactness {:.6f}/ Separateness {:.6f}'.format(loss_pixel.item(), compactness_loss.item(), separateness_loss.item()))
    print('Memory_items:')
    print(m_items)
    print('----------------------------------------')

In [ ]:
print('Training is finished')
# Save the model and the memory items
torch.save(model, os.path.join(log_dir, 'model.pth'))
torch.save(m_items, os.path.join(log_dir, 'keys.pt'))

In [ ]:
sys.stdout = orig_stdout
f.close()